1. most used platform
2. no. of failed answers per conversation and no. of questions that did not satisfy user (with feedback no)
3. questions that the bot couldn't answer/ got neg feedback  
4. character length of user-requests 
5. perform NER and find commonly asked topics 
6. Do coreferences exist?

In [1]:
import json
import pprint
import re

In [2]:
path = 'data/grouped_chats.json'

with open(path, 'r') as file:
    data = json.load(file)
    i = 0
    for key, value in data.items():
        print(len(value))
        pprint.pprint(value)
        i += 1
        if i == 5:
            break

24
[{'fromBot': False,
  'id': 'dbfaba55-ecc0-42a3-b2c7-1daa39fb36b3_1553869389296_request',
  'request': {'messageData': [{'payload': 'start'}],
              'messageType': 'parameter',
              'platform': 'Web',
              'userId': 'dbfaba55-ecc0-42a3-b2c7-1daa39fb36b3'},
  'response': nan,
  'timestamp': 1553869389296.0,
  'userId': 'dbfaba55-ecc0-42a3-b2c7-1daa39fb36b3'},
 {'fromBot': True,
  'id': 'dbfaba55-ecc0-42a3-b2c7-1daa39fb36b3_1553869389296_response',
  'request': nan,
  'response': [{'messageData': [{'buttons': [],
                                 'fields': [],
                                 'image': 'http://i.imgur.com/044oUKK.png',
                                 'smartReplies': [],
                                 'text': 'I am the DBpedia Bot'}],
                'messageType': 'text'},
               {'messageData': [{'buttons': [],
                                 'fields': [],
                                 'image': 'http://i.imgur.com/044oUKK.png',


 {'fromBot': True,
  'id': '448e21ab-e862-42d3-b106-50608fea9381_1515987062530_response',
  'request': nan,
  'response': [{'messageData': [{'buttons': [],
                                 'fields': [],
                                 'image': 'http://i.imgur.com/044oUKK.png',
                                 'smartReplies': [{'slackStyle': 'default',
                                                   'title': 'Similar',
                                                   'uri': 'load-similar__http://dbpedia.org/resource/Albert_Einstein__Albert '
                                                          'Einstein'},
                                                  {'slackStyle': 'default',
                                                   'title': 'Related',
                                                   'uri': 'load-related__http://dbpedia.org/resource/Albert_Einstein__Albert '
                                                          'Einstein'}],
                              

  'response': [{'messageData': [{'buttons': [],
                                 'fields': [],
                                 'image': 'http://i.imgur.com/044oUKK.png',
                                 'smartReplies': [],
                                 'text': "Here's some info I found:"}],
                'messageType': 'text'},
               {'messageData': [{'buttons': [{'buttonType': 'link',
                                              'slackStyle': 'default',
                                              'title': 'View Map',
                                              'uri': 'http://nominatim.openstreetmap.org/search?q=Hawaii&addressdetails=1&extratags=1&limit=1'},
                                             {'buttonType': 'param',
                                              'slackStyle': 'default',
                                              'title': 'Basic Information',
                                              'uri': 'entity-information__http://dbpedia.org/reso

  'request': {'messageData': [{'payload': 'feedback__yes__03b12284-8cc5-47e2-9786-2319ee6b32ea_1553665556350'}],
              'messageType': 'parameter',
              'platform': 'Web',
              'userId': '03b12284-8cc5-47e2-9786-2319ee6b32ea'},
  'response': nan,
  'timestamp': 1553665560344.0,
  'userId': '03b12284-8cc5-47e2-9786-2319ee6b32ea'},
 {'fromBot': True,
  'id': '03b12284-8cc5-47e2-9786-2319ee6b32ea_1553665560344_response',
  'request': nan,
  'response': [{'messageData': [{'buttons': [],
                                 'fields': [],
                                 'image': 'http://i.imgur.com/044oUKK.png',
                                 'smartReplies': [],
                                 'text': 'Glad that you like it.'}],
                'messageType': 'text'}],
  'timestamp': 1553665560358.0,
  'userId': '03b12284-8cc5-47e2-9786-2319ee6b32ea'},
 {'fromBot': False,
  'id': '03b12284-8cc5-47e2-9786-2319ee6b32ea_1553665572342_request',
  'request': {'messageData

 {'fromBot': True,
  'id': '03b12284-8cc5-47e2-9786-2319ee6b32ea_1553695238813_response',
  'request': nan,
  'response': [{'messageData': [{'buttons': [{'buttonType': 'link',
                                              'slackStyle': 'default',
                                              'title': 'View in Wikipedia',
                                              'uri': 'http://en.wikipedia.org/wiki/German_Empire'},
                                             {'buttonType': 'link',
                                              'slackStyle': 'default',
                                              'title': 'View in DBpedia',
                                              'uri': 'http://dbpedia.org/resource/German_Empire'},
                                             {'buttonType': 'param',
                                              'slackStyle': 'default',
                                              'title': 'Learn More',
                                              'uri': 'le

In [3]:
# most used platform 
from collections import Counter

platform_count = Counter()
err = 0

for key, val in data.items():
    msg = val[0]
    if type(msg['request']) == float: # check if msg.request = nan
        err += 1
#         pprint.pprint(msg)
    else:
        platform_count[msg['request']['platform']] += 1
            
print(platform_count)
print(err)

Counter({'Web': 9078, 'Slack': 4, 'Facebook Messenger': 1})
1


In [4]:
# no. of failed answers per conversation and questions that bot couldn't answer
from collections import defaultdict

failed_qid_per_user = defaultdict(set)
failed_qid = set()
num_requests = 0 # includes START payload from user  
passed = 0
feedback_asked = 0 

for uid, conv in data.items():
    for msg in conv: # msg object
        if not msg['fromBot']: 
            num_requests += 1
            for obj in msg['request']['messageData']:
                if 'payload' in obj and 'feedback__no__' in obj['payload']:
                    failed_qid_per_user[uid].add(obj['payload'].replace('feedback__no__', ''))
                    failed_qid.add(obj['payload'].replace('feedback__no__', ''))
                    break
                elif 'payload' in obj and 'feedback__yes__' in obj['payload']:
                    passed += 1
                    break
                
        else:
            if type(msg['response']) != float:
                for obj in msg['response']:
                    if obj['messageType'] == 'smart_reply':
                        feedback_asked +=1
                        break


print('num_requests: ', num_requests)
print('feedback_no: ', len(failed_qid))
print('convos with failed responses: ', len(failed_qid_per_user))
print('feedback_yes: ', passed)
print('feedback_asked: ', feedback_asked)

num_requests:  45797
feedback_no:  4155
convos with failed responses:  1982
feedback_yes:  3406
feedback_asked:  28953


In [5]:
# questions that the bot couldn't answer
unanswered_ques = []

for uid, conv in data.items():
    if uid in failed_qid_per_user:
        for msg in conv: # msg object
            if not msg['fromBot']:
                if msg['id'].replace('_request', '') in failed_qid_per_user[uid]:
                    for obj in msg['request']['messageData']:
                        print(obj)
                        if 'text' in obj:
#                             print(obj)
                            unanswered_ques.append(obj['text'])
                            #break

print(len(unanswered_ques))

{'text': 'Who is George MacDonald Church ?'}
{'text': 'Who is Macron ?'}
{'payload': 'dbpedia-about'}
{'text': 'What is the plot of Titanic?'}
{'payload': 'dbpedia-about'}
{'text': 'who is sachin>'}
{'text': 'what is anthem of india?'}
{'text': 'what is life'}
{'text': 'How many bones does a cat have?'}
{'text': 'How to care a newborn?'}
{'text': 'How to care for newborn?'}
{'text': 'Where was Angela Merkel born '}
{'text': 'who is Angela Merkel?'}
{'text': 'Tell me something about Rachmaninov'}
{'text': 'what is konami'}
{'text': 'hello'}
{'text': 'where was george clooney born?'}
{'text': 'who is the mayor of Varese?'}
{'text': 'What is the German Empire?'}
{'text': 'who is einstein'}
{'text': 'Programming Language'}
{'text': 'how many members does the Beatles have?'}
{'text': 'The top biggest cities'}
{'text': 'The most difficult problem '}
{'text': 'heloo'}
{'payload': 'help'}
{'payload': 'dbpedia-about'}
{'payload': 'help'}
{'payload': 'dbpedia-contribute'}
{'payload': 'entity-inf

{'text': 'what is bim?'}
{'text': 'what is a building?'}
{'text': 'Who is Bill Gates?'}
{'text': 'yes'}
{'text': 'who is lee smolin? '}
{'text': 'what is Pluto?'}
{'text': 'tell me some Kunwinjku words'}
{'text': 'Kunwinjku'}
{'text': 'What is Kunwinjku?'}
{'text': 'where is Gunbalanya'}
{'text': 'what is a chatbot'}
{'text': 'why are you so slow'}
{'text': 'what is capital of japan'}
{'text': 'Where is Saber?'}
{'text': 'who lost the presidential race against donald trump'}
{'text': 'what is my question?'}
{'text': 'who is majid samiee'}
{'text': 'Where is Mao Zedong from?'}
{'text': 'where is Shandong'}
{'text': 'what is numpy'}
{'text': 'Who is Diego Madona'}
{'text': 'Who is Diego Maradona?'}
{'text': 'Who is Angela Merkel'}
{'text': 'is Hindutani Fijian similar to Hindi'}
{'text': 'Who is Narendra Modi?'}
{'text': 'Where is Senja?'}
{'text': 'when did shackleton visit the antarctic'}
{'text': 'when did ernest shackleton travel to antarctica'}
{'text': 'who was ernest shackleton'}


In [6]:
# get convos as request-response pairs, irrespective of userId  
utter = dict() # { qid : {req : [response]}}

for uid, conv in data.items():
    for msg in conv: # msg object
        if type(msg['id']) != float and '_request' in msg['id']: # request-from-user, add to dict
            qid = msg['id'].replace('_request', '')
            req_ob = msg['request']['messageData'][0]
            for key in req_ob:
                request = req_ob[key]
                obj = {request : []}
                utter[qid] = obj # every req-response pair is marked by a unique id 
                
        elif type(msg['id']) != float:
            resp_id = msg['id'].replace('_response', '')
            resp_list = msg['response']
            response = []
            if resp_id in utter:
                for obj in resp_list:
                    for text_ob in obj['messageData']:
                        if 'text' not in text_ob:
                            response.append(text_ob['title'])
                        else:
                            response.append(text_ob['text'])
                       
                for key in utter[resp_id]:
                    utter[resp_id][key].extend(response)
                    
with open('data/utterances.json', 'w') as outfile:
    json.dump(utter, outfile)


In [7]:
print(len(utter))

45797


In [16]:
# character length of user-requests 
from collections import Counter
reqlen = Counter() # {len - no. of requests having that len}

for id in utter:
    for req in utter[id]:
        if req != 'start' and 'feedback' not in req:
            reqlen[len(req)] += 1
pprint.pprint(reqlen)
print(max(reqlen.keys()))
print(min(reqlen.keys()))

Counter({13: 1848,
         16: 1585,
         20: 1263,
         15: 1210,
         63: 1210,
         17: 1144,
         18: 1126,
         12: 989,
         22: 956,
         14: 916,
         19: 894,
         2: 859,
         54: 755,
         3: 752,
         21: 745,
         23: 666,
         5: 631,
         24: 615,
         11: 593,
         4: 588,
         25: 576,
         26: 533,
         30: 521,
         27: 517,
         31: 499,
         29: 498,
         28: 496,
         32: 389,
         6: 383,
         7: 371,
         9: 356,
         8: 352,
         44: 343,
         10: 336,
         33: 328,
         34: 306,
         47: 303,
         35: 281,
         36: 235,
         37: 225,
         38: 221,
         39: 196,
         40: 177,
         56: 166,
         42: 157,
         41: 152,
         61: 136,
         43: 124,
         72: 112,
         46: 103,
         74: 99,
         1: 87,
         68: 84,
         52: 82,
         48: 77,
         45: 76,


#### analyse requests - perform NER and find commonly asked topics

In [7]:
# load data 
with open('data/utterances.json', 'r') as file:
    utter = json.load(file)

print(len(utter))

45797


In [11]:
# data cleaning
def data_clean():
    clean_text = []
    for id in utter:
        for req in utter[id]:
            if req != 'start' and 'feedback' not in req:
                req = re.sub(r'http://dbpedia.org/resource/', ' ', req)
                req = re.sub(r"-|_|'|/|!", ' ', req) 
                req = re.sub(r' +', ' ', req)
                print(req)
                clean_text.append(req)
    return clean_text

preprocessed_text = data_clean()

Who is George MacDonald Church ?
Who is Macron ?
Who is Emmanuel Macron ?
Who is Einstein ?
entity information Albert Einstein
WHER IS PARIS ?
WHERE IS PARIS?
WHO IS GEORGE CHURCH?
WHO IS MACRON?
Who is Yao Ming swife？
entity information Albert Einstein
entity information Planet
get location Berlin
dbpedia about
entity information Albert Einstein
learn more Albert Einstein Albert Einstein
load similar Albert Einstein Albert Einstein
get location Berlin
entity information Planet
entity information Berlin
learn more Planet Planet
load similar Planet Planet
load related Planet Planet
dbpedia about
entity information Albert Einstein
get location Berlin
learn more Albert Einstein Albert Einstein
who is obama
get location Berlin
where is newyork
Where is Beijing
Where is Hawaii
What is lenovo
What is Lenovo
entity information Planet
get location Berlin
Who directed Edge of Tomorrow?
What is the plot of Titanic?
dbpedia about
who is sachin>
who is sachin?
hello
what you can do?
where is berli

hello
no
no 
what is google
what is dbpedia
where is Brazil capital
brazil capital
Germany capital
first to claimb mont everest
first to claim mont everest
monte evereste
monite everiste
barack obama wife
learn more Michelle Obama Michelle Obama
load similar Michelle Obama Michelle Obama
load related Michelle Obama Michelle Obama
obama wife
where is Paris
paris
who killed Jesse James?
Hi bot
Who is Guderian
Who is bill gates
Who is Guderian ?
Where is the capital of United Arab Emirates
Give me the capital of China
Give me the capital of Germany
entity information Albert Einstein
entity information Planet
Who is Jinping Xi
Who is the predident of US
get location Berlin
Where is Paris
entity information Albert Einstein
tell me who is bill gates
Hello
How are you?
Stop asking me whether your responses were helpful
Who is adolf hitler?
Do you like him?
What is misslick?
Anal
dbpedia about
Who is adolf hitler
dbpedia about
entity information Albert Einstein
entity information Albert Einste

where is agira
where is agira sicilia
where is pleviersingel s gravenhage
where is pleviersingel den haag netherlands
where is pleviersingel in den haag in the netherlands
Do you know anything about me?
No
What can you do?
Who is Einstein>
Who is Einstein?
When did Einstein die?
Where did Einstein die?
Where did Einsten die?
hi
sort of
no
yes
can you understand me
what is your name?
i feel sad
i failed an exam yesterday
i failed the exam cause i didnt study
what is empathy?
are you empathetic
because i would like an empathetic bot
yes, that is the real reason
now answer my question
are you empathetic
i will prefer if you were empathetic
no
stop asking me if it was helpful
stop saying sorry so much
who are you?
yeah
entity information Albert Einstein
How many inhabitants has Stuttgart?
What is the area of Stuttgart?
What is the distance from Stuttgart to Berlin?
What is the distance from Munich to Bremen?
What is the distance from Munich to Stuttgart?
What language is spoken in Brazil?


what is tea?
where is Berlin?
How to make a good espresso cup?
Is coffee healthy ?
Is coffee cold?
what is the brew temperature of coffee ?
who are u?
what is your name?
yu
entity information Albert Einstein
Which is the highest selling Pink Floyd album?
hello
now time
what s the time
helpful?
maybe
yes
soga
what s the weather?
what is your name
are you hugury
can you introduce your country?
yes
introduce your country please
ls
oh my gad 
What is life?
What are thoughts?
How does a car work?
Who is Einstein?
entity information Albert Einstein
entity information Planet
哈哈
 what is 中国
dbpedia about
get location Berlin
what is a circle?
what is a career?
what is sustainability?
dbpedia about
entity information Albert Einstein
dbpedia about
entity information Planet
Who are you?
How large is the earth?
How large is Jupiter?
Who is the wife of Donald Trump?
What is the capital of Germany?
What is oldest child of Donald Trump?
IMEI
IMEC
What is IMEC
Where is Berlin
entity information Berlin


WHAT IS THE CITY OF GAROUA ?
WHAT IS A MOUNTAIN OF CAMEROON ?
WHO AFRICAN WIN NOBEL PRIZE ?
AFRICAN WIN NOBEL PRIZE ?
which African won the Nobel Prize?
WHO IS WATTAO ?
WHO IS Issiaka Ouattara ?
dbpedia about
dbpedia contribute
entity information Albert Einstein
How let the dogs out?
dbpedia about
entity information Albert Einstein
dbpedia about
How much HP has the 1.3 DDIS
How many tires has a car?
How many engines has the falcon heavy?
What is SpaceX?
entity information Albert Einstein
entity information Albert Einstein
What is the Kölnder Dome?
What is the Kölner Dome?
What is the Eifeltower?
Eifel tower
bitstar
bitstars GmbH
dbpedia about
Who is mr. putin
Who is mr. putin
Who is Putin
How much is the fish
What is a Kickflip
entity information Planet
how much data on internet?
How much amount of data does exist in the whole internet?
No
You gave me adds:(
Who is Andrés Gimeno?
Who is John McEnroe?
Give me the capital of Spain.
What is dbpedia?
Which is the capital of Germany?
Give m

what is finance
what is stock market
tesla 
elon musk
who is elon musk
check service dbpedia
who is tesla s owner
yes
hi
tesla stock
entity information Albert Einstein
who is steven jobs
tell me about chiranjeevi
where is heathrow
heathrow airport
london heathrow
who is einstein
where is berlin
who is obama
what s dbpedia
is dbpedia down?
Where is France s capital?
spanish dbpedia
where s spain capital
where is barcelona
who lives in barcelona
entity information Albert Einstein
dbpedia contribute
dbpedia about
dbpedia about
entity information Albert Einstein
countries of europe
good evening 2 2
Who is Obama
Who is Obama?
entity information Albert Einstein
Who is Newton?
dbpedia contribute
Who is Einstein?
What is a planet
What is Jupiter
Jupiter
ncdjozncdjozndjozcndjozcndjozcndozcndozcnojdzcnozcndjzo
What is a car?
what is a bicycle?
How many wheels does a bicycle have?
How will you improve?
Who are you?
Who made you?
What are their names?
is dbpedia down?
What is your purpose?
Who are

entity information Planet
tell me about einstein
who is churchill?
learn more Winston Churchill Winston Churchill
load related Winston Churchill Winston Churchill
load similar Winston Churchill Winston Churchill
Hi
Where is Ufa?
What is your name?
What s the weather in NY?
How old are you?
Are you here?
How old are you?
Who is creator of electricity?
colours
fc nantes
what is Fc Nantes ?
entity information Albert Einstein
Nantes
What is Nantes ?
will you add something addition addition of numbers
entity information Albert Einstein
entity information Planet
dbpedia about
get location Berlin
what is "linked data"
what are you?
do you like me?
Because you want to fuck me
yes
so what do you like?
no
I ask you
no i ask you know
now
no
dbpedia about
what is the most consulted topic? 
what is the best newmetal band?
learn more New Metal New Metal
load related New Metal New Metal
what is the best nu metal band?
Is Berlin more populated than Rome? 
get location Berlin
entity information Planet


Who is Einsein
Netflix tv series of 2016
get me the list of movies directed by tarantino
dbpedia about
dbpedia about
check service dbpedia
entity information Planet
Who s Juhani Salovaara?
Who s Albert Einstein?
Who s Juhani Salovaara?
where is switzerland
?
what is the capital of croatia
who is a hurensohn?
who has the longest dick?
something wrong boi?
hello darkness my old friend
what is swag?
how to get rich?
are you high?
who is antonio bozic?
tell me the real numbers
ronaldo
cristiano ronaldo
christiano ronaldo
chile
http protocol
FTP protocol
Ronaldo
important event in chile
what is america?
what are vowels
is berlin in germany
berlin and germany
einsterin scholar
hi
what is time
define time
what is the definition of time
entity information Planet
abou
where is barcelona
entity information Barcelona
hightest mountain
한국의 수도는?
what is the height of himalayas
dbpedia about
entity information Planet
ok tell me some good habits
Do I need Visa for UK?
What is the cost of work permit 

entity information Planet
who is elsisi?
citation of spinoza
Who is Ho Chi Minh
Who is Obama s wife?
entity information Albert Einstein
who is barack obama?
who is behind DBpedia?
you do not have right
entity information Albert Einstein
entity information Planet
get location Berlin
entity information Albert Einstein
what are you studying
entity information Albert Einstein
not ok
Who is the us president who stayed in office for the shortest time
Who is the usa president who stayed in office for the shortest time
can you play a rhyme for me
where i live
what is your name do
What is a kitchen ?
Who is emmanuel macron?
entity information Albert Einstein
can you tell me how to get to sesame street?
but where the fuck is it?
I feel like you re not really listening to me...
You repeat my words but you don t seem to understand them
see this is what I m talking about, you re not there for me...
look I think we need to see other people
had your breakfast
switch off the future
entity information 

chief minikster of ap
hello bad morning
from which place are you coming from
good morning good morning
how many languages in india
what is line
how many states in india
what is name your
what is a neural network
tell me about a neural network
what does an aeroplane do
my name is mr due to
please explain me about body parts of a human being
how many colours are there
tell me the story of thirsty crow
are you feeling sleepy
how many district in up
shutdown the computer
of show me the image of apple
flowers
which is the richest country in the world
tell me about potato
what is the weather in pakistan
chief guest of event
tell about india
When was the first sonic game released?
set alarm at 10:30 am
staff
gps location
telangana
i want to see a flower
who discovered telephone
box strapping machine automatic 1 second order audio signal lakh to the back to school
who is this
more rhymes
what is your grandfather name
how many letters are there in
images of animals
who invented the thermometer


In [18]:
# perform NER
import requests
from collections import defaultdict


def get_surface_forms_schema(response, entity_map):
    sf_text = {} # surface_form:schema 
    if 'Resources' in response:
        for res in response['Resources']:
            surface_form = res['@surfaceForm']
            types = res['@types']
            if len(types) != 0:
                type_dict = dict_from_str(types) 
                if 'Schema' in type_dict:
                    for sch in type_dict['Schema']:
                        entity_map[sch].append(surface_form)
                        sf_text[surface_form] = sorted(type_dict['Schema'])[0]
    return sf_text

def dict_from_str(string): # str: 'abc:def,fgh:ijk,..'
    d = defaultdict(list)
    for element in string.split(','):
        el = element.split(':')
        d[el[0].strip()].append(el[1].strip())
    return d

def extract_entities(replace=True):
    entity_map = defaultdict(list) # schema: [entity] -> {person: [name1, ..], thing: [thing1,..]}
    i = 0
    ner_text = []
    for text in preprocessed_text:
        i +=1 
        if i%100==0:
            print(i)
        headers = {'Accept': 'application/json'}
        data = {"text":text, 'confidence': '0.35'}
        response = requests.post('http://model.dbpedia-spotlight.org/en/annotate', headers=headers, data=data)
        if response.status_code == requests.codes.ok:
            json_res = response.json()
            sf_schema = get_surface_forms_schema(json_res, entity_map)
            if replace: # replace the surface form with Schema in string
                for sf in sf_schema:
                    if not text.startswith(sf) and sf.lower() != 'dbpedia':
                        text = text.replace(sf, sf_schema[sf])
                        ner_text.append(text)

    return entity_map, ner_text

entity_map, ner_text = extract_entities(True)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [19]:
print(ner_text)

['Who is Person Church ?', 'Who is Person ?', 'Who is Person ?', 'Who is Person ?', 'entity information Person', 'WHO IS Person?', 'Who is Person swife？', 'entity information Person', 'get location City', 'entity information Person', 'learn more Person Person', 'load similar Person Person', 'get location City', 'entity information City', 'entity information Person', 'get location City', 'learn more Person Person', 'who is Person', 'get location City', 'where is AdministrativeArea', 'Where is Place', 'Where is AdministrativeArea', 'What is Organization', 'What is Organization', 'get location City', 'Who directed MusicGroup of Tomorrow?', 'where is City?', 'entity information Country', 'entity information Person', 'birthdate of Person?', 'entity information Person', 'load similar Person Person', 'load related Person Person', 'DBPEDIA Event', 'amitab Place', 'What is the Language name of a cat', 'How old did Person get?', 'How high is the Place?', 'entity information Person', 'get locatio

In [14]:
print(entity_map)

defaultdict(<class 'list'>, {'Person': ['George MacDonald', 'Macron', 'Emmanuel Macron', 'Einstein', 'Albert Einstein', 'GEORGE CHURCH', 'Yao Ming', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'obama', 'Edge', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'rajanikanth', 'abdul kalam', 'elon musk', 'kamal hasan', 'Einstein', 'Albert Einstein', 'Angela Merkel', 'Angela Merkel', 'Albert Einstein', 'hideo kojima', 'george clooney', 'george washington', 'george washington', 'Albert Einstein', 'Alfred', 'einstein', 'Einstein', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'Albert Einstein', 'sara tancredi', 'sara', 'einstein', 'einstein', 'obama', 'obama', 'obama', 'Albert Einstein', 'barack obam

In [24]:
schema_entity_map = entity_map
with open('data/entity_map.json', 'w') as f:
    json.dump(entity_map, f)

In [25]:
# schema of recognised entities 
for k,v in entity_map.items():
    print(k, len(v))

Person 5661
RadioStation 53
Organization 1105
Place 5293
City 1717
CreativeWork 2573
AdministrativeArea 225
MusicGroup 349
Country 1703
Event 79
Language 161
Mountain 24
RiverBodyOfWater 30
BodyOfWater 37
Movie 120
MusicAlbum 13
LakeBodyOfWater 6
SportsTeam 112
TelevisionStation 15
LandmarksOrHistoricalBuildings 24
EducationalOrganization 105
CollegeOrUniversity 97
WebPage 46
SportsEvent 15
Product 37
BankOrCreditUnion 18
Hospital 1
Book 37
Continent 14
Airport 14
School 8
Park 7
MusicRecording 4
TVEpisode 1
SkiResort 1


In [34]:
# write to file 
with open('data/ner_requests.txt', 'w') as f:
    f.write(str(ner_text))

#### Do coreferences exist?

In [ ]:
import spacy, ast

nlp = spacy.load('en_core_web_md')

In [23]:
# check the POS tag for pronoun

coref = []
has_you = []
has_me = []
pronouns = set()

# with open('ner_requests.txt', 'r') as f:
#     lines = f.read()
for line in preprocessed_text:
    if 'tell me' in line.lower():
        line = line.lower().replace('tell me', 'give')
        #print(line)
    doc = nlp(line)
    for token in doc:
        if token.pos_=='PRON' and token.tag_=='PRP':
            pronouns.add(token.text.lower())
            coref.append(line)
            if token.text.lower() == 'you' or token.text.lower() == 'your' or token.text.lower() == 'yours':
                has_you.append(line)
            if token.text.lower() == 'i' or token.text.lower() == 'my' or token.text.lower() == 'mine':
                has_me.append(line)

print('# coref: ', len(coref))
print('# times chatbot was referred: ', len(has_you))
print('# times user refers himself: ', len(has_me))
print('# reqs: ', len(lines))        
            
        

# coref:  4491
# times chatbot was referred:  2373
# times user refers himself:  1038
# reqs:  349132


In [26]:
# kind of pronouns/coreferences used
print(pronouns)

{'ther', 's', 'ut', 'hw', 'levis', 'yo', 'them', 'hwo', 'he', 'yours', 'em', 'it', 'hp', 'whait', 'we', 'yourself', 'y', 'they', 'her', 'my', 'm', 'him', 'she', 'i', 'me', 'us', 'u', 'c', 'myself', 'you'}


In [19]:
print(coref)

['what you can do?', 'who created you', 'what day is it today', 'are you happy', 'who are you', 'Google is it a company ?', 'where can I find the mapping tool', 'give me the capital of germany', 'where can I find the mapping tool', 'how are you?', 'can you give what is an apple', 'Who are you?', 'What can I ask you?', 'What can I ask you?', 'How do you work?', 'What are you?', 'How do you work?', 'How old is he?', 'i am fine how are you', 'i am fine how are you', 'How are you today?', 'What will you eat for supper today?', 'You may help me choosing a recipe for supper', 'You may help me choosing a recipe for supper', 'What do you know about Radboud University?', 'Am I annoying?', 'Are you real?', 'Show me a Claude Monet painting', 'where i am', 'do you know, where is indonesia', 'could you give about the best disguises mortadelo has ever used?', 'could you give please who is the mortadelo companion?', 'what can you do ', 'how are you ?', 'how are you ?', 'can you show me the image of n

#### language other than english

In [24]:
!pip install spacy-langdetect

In [37]:
# TODO: check if the results match with https://pypi.org/project/langdetect/
# TODO: try sentence-wise detection?
from spacy_langdetect import LanguageDetector
from collections import defaultdict
#nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

lang_req = defaultdict(list)

for line in preprocessed_text:
    doc = nlp(text)
    #print(doc._.language['language'])
    # document level language detection. Think of it like average language of the document!
    lang_req[doc._.language['language']].append(line)
    

for k,v in lang_req.items():
    print(k, len(v))

en 30263
